In [61]:
%reset -f
##THis code processes the MAIAC MCD19A2 1km AOD product from both AQUA nad TERRA Datasets
import os
import re
import pyproj
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC
FILE_NAME = 'MCD19A2.A2018120.h26v05.061.2023117102707.hdf'
hdf = SD(FILE_NAME, SDC.READ)

In [103]:
time_array=[]
import datetime
from datetime import time
list(hdf.attributes())
orbit_time= hdf.attributes()['Orbit_time_stamp']
orbit_time
orbit_time_list = orbit_time.split()
time_stamps = []
for timestamp in orbit_time_list:
    year = int(timestamp[0:4])
    julian_day = int(timestamp[4:7])
    hour = int(timestamp[7:9])
    min = int(timestamp[9:11])
    gregorian_date = datetime.datetime(year, 1, 1) + datetime.timedelta(days=julian_day - 1)
    year = gregorian_date.year
    month = gregorian_date.month
    day = gregorian_date.day
    time_stamps.append(gregorian_date.replace(hour=hour, minute=min))
    print(f"Year: {year}, Month: {month}, Day: {day}, Hour: {hour}, Minute: {min}")
    time_array = np.array(time_stamps, dtype='datetime64')
print(time_array)

Year: 2018, Month: 4, Day: 30, Hour: 3, Minute: 25
Year: 2018, Month: 4, Day: 30, Hour: 5, Minute: 0
Year: 2018, Month: 4, Day: 30, Hour: 5, Minute: 5
Year: 2018, Month: 4, Day: 30, Hour: 6, Minute: 40
['2018-04-30T03:25:00.000000' '2018-04-30T05:00:00.000000'
 '2018-04-30T05:05:00.000000' '2018-04-30T06:40:00.000000']


In [63]:
### DATA FIELD PROCESS AND REPROJECT ###
DATAFIELD_NAME1 = 'Optical_Depth_055'
aod550=None
# Read dataset.
aod550_3D = hdf.select(DATAFIELD_NAME1)
aod550_shape = aod550_3D.info()[2]
aod550_shape
num_orbits = aod550_shape[0]
num_orbits
# Read attributes.
attrs = aod550_3D.attributes(full=1)
attrs
lna=attrs["long_name"]
long_name = lna[0]
vra=attrs["valid_range"]
valid_range = vra[0]
fva=attrs["_FillValue"]
_FillValue = fva[0]
sfa=attrs["scale_factor"]
scale_factor = sfa[0]        
ua=attrs["unit"]
units = ua[0]
aoa=attrs["add_offset"]
add_offset = aoa[0]
for orbit_num in range(num_orbits):
    aod=aod550_3D[orbit_num, :, :].astype(np.double)
    invalid = np.logical_or(aod < valid_range[0], aod > valid_range[1])
    invalid = np.logical_or(invalid, aod == _FillValue)
    aod[invalid] = np.nan
    aod = (aod - add_offset) * scale_factor
    aod = np.ma.masked_array(aod, np.isnan(aod))
    ## save as seperate variable##
    var_name = f"aod550_{orbit_num}"
    exec(f"{var_name}=aod")
    print(f"\n{var_name}") 
    # Stack 
    if aod550 is None:
        aod550 = aod[np.newaxis, :,:]
    else:
        aod550 = np.concatenate([aod550, aod[np.newaxis,:,:]], axis=0)
print(aod550.shape)


aod550_0

aod550_1

aod550_2

aod550_3
(4, 1200, 1200)


In [64]:
DATAFIELD_NAME2 = 'Optical_Depth_047'
aod470=None
hdf = SD(FILE_NAME, SDC.READ)
# Read dataset.
aod470_3D = hdf.select(DATAFIELD_NAME2)
aod470_shape = aod470_3D.info()[2]
num_orbits = aod470_shape[0]
# Read attributes.
attrs = aod470_3D.attributes(full=1)
lna=attrs["long_name"]
long_name = lna[0]
vra=attrs["valid_range"]
valid_range = vra[0]
fva=attrs["_FillValue"]
_FillValue = fva[0]
sfa=attrs["scale_factor"]
scale_factor = sfa[0]        
ua=attrs["unit"]
units = ua[0]
aoa=attrs["add_offset"]
add_offset = aoa[0]
for orbit_num in range(num_orbits):
    aod1=aod470_3D[orbit_num, :, :].astype(np.double)
    invalid = np.logical_or(aod1 < valid_range[0], aod1 > valid_range[1])
    invalid = np.logical_or(invalid, aod1 == _FillValue)
    aod1[invalid] = np.nan
    aod1 = (aod1 - add_offset) * scale_factor
    aod1 = np.ma.masked_array(aod1, np.isnan(aod1))
    ## save as seperate variable##
    var_name = f"aod470_{orbit_num}"
    exec(f"{var_name}=aod1")
    print(f"\n{var_name}") 
    # Stack 
    if aod470 is None:
        aod470 = aod1[np.newaxis, :,:]
    else:
        aod470 = np.concatenate([aod470, aod[np.newaxis,:,:]], axis=0)
print(aod470.shape)


aod470_0

aod470_1

aod470_2

aod470_3
(4, 1200, 1200)


In [65]:
DATAFIELD_NAME3 = 'AOD_QA'
aodqa=None
hdf = SD(FILE_NAME, SDC.READ)
# Read dataset.
aodqa_3D = hdf.select(DATAFIELD_NAME3)
# Read dataset.
aodqa_shape = aodqa_3D.info()[2]
num_orbits = aodqa_shape[0]
# Read attributes.
attrs = aodqa_3D.attributes(full=1)
lna=attrs["long_name"]
long_name = lna[0]
vra=attrs["valid_range"]
valid_range = vra[0]
fva=attrs["_FillValue"]
_FillValue = fva[0]
for orbit_num in range(num_orbits):
    qa=aodqa_3D[orbit_num, :, :].astype(np.double)
    invalid = np.logical_or(qa < valid_range[0], qa > valid_range[1])
    invalid = np.logical_or(invalid, qa == _FillValue)
    qa[invalid] = np.nan
    qa = np.ma.masked_array(qa, np.isnan(qa))
    ## save as seperate variable##
    var_name = f"aodqa_{orbit_num}"
    exec(f"{var_name}=qa")
    print(f"\n{var_name}") 
    if orbit_num == 0:
        aodqa = qa[np.newaxis, :,:]
    else: 
        aodqa = np.concatenate([aodqa, qa[np.newaxis,:,:]], axis=0)#aodqa.shape
print(aodqa.shape)
def int_to_binary(x):
    if np.isnan(x):
        return np.nan  # Return NaN for NaN values
    return format(int(x), '016b')  # Convert integers to 16-bit binary strings

vectorized_int_to_binary = np.vectorize(int_to_binary, otypes=[object])

# Convert aodqa to binary
aodqa_bin = vectorized_int_to_binary(aodqa)

print(f"Shape of aodqa_bin: {aodqa_bin.shape}")


aodqa_0

aodqa_1

aodqa_2

aodqa_3
(4, 1200, 1200)
Shape of aodqa_bin: (4, 1200, 1200)


In [54]:
## Important Note ## PLEASE READ
#Each QA is a 16 bit file 
#4.4 AOD QA definition for MCD19A2 (16-bit unsigned integer)
#Bits Definition
#0-2 Cloud Mask
#000 --- Undefined
#001 --- Clear
#010 --- Possibly Cloudy (detected by AOD filter)
#011 --- Cloudy (detected by cloud mask algorithm)
#101 --- Cloud Shadow
#110 --- Hot spot of fire
#111 --- Water Sediments
#3-4 Land Water Snow/ice Mask
#00 --- Land
#01 --- Water
#10 --- Snow
#11 --- Ice
#5-7 Adjacency Mask
#000 --- Normal condition/Clear
#001 --- Adjacent to clouds
#010 --- Surrounded by more than 8 cloudy pixels
#011 --- Adjacent to a single cloudy pixel
#100 --- Adjacent to snow
#101 --- Snow was previously detected in this pixel
#8-11 QA for AOD
#0000 --- Best quality
#0001 --- Water Sediments are detected (water)
#0011 --- There is 1 neighbor cloud
#0100 --- There is >1 neighbor clouds
#0101 --- No retrieval (cloudy, or whatever)
#0110 --- No retrievals near detected or previously detected snow
#0111 --- Climatology AOD: altitude above 3.5km (water) and 4.2km (land)
#1000 --- No retrieval due to sun glint (water)
#1001 --- Retrieved AOD is very low (<0.05) due to glint (water)
#1010 --- AOD within +-2km from the coastline (may be unreliable)
#1011 --- Land, research quality: AOD retrieved but CM is possibly cloudy
#12 Glint Mask
#0 --- No glint
#1 --- Glint (glint angle < 40)
#13-14 Aerosol Model
#00 --- Background model (regional)
#01 --- Smoke model (regional)
#10 --- Dust model
#15 Reserved

In [53]:
# Please READ
#From the combination of different 16 bit AODQA, we will be considering the following combinations to be valid 
## 0000000000000001
## 0000000000001001
## 0010000000000001
## 0010000000001001
#Criteria for Validity:
#Bits 0-2 (Cloud Mask): Should be 001 (Clear)
#Bits 3-4 (Land Water Snow/Ice Mask): Should be 00 (Land) or 01 (Water)
#Bits 5-7 (Adjacency Mask): Should be 000 (Normal condition/Clear)
#Bits 8-11 (AOD level and related flags): Should be 0000 (Best QA)
#Bit 12 (No glint): Should be 0
#Bits 13-14 (Aerosol Model): Should be 00 or 01
#Bit 15 (Reserved): Should be 0
#Let's break down the criteria and see if these patterns cover all possibilities:
#Bits 0-2 (Cloud Mask): Must be 001 (Clear) - This is consistent in all patterns.
#Bits 3-4 (Land/Water): Can be 00 (Land) or 01 (Water) - Your patterns cover both.
#Bits 5-7 (Adjacency): Must be 000 (Normal condition/Clear) - This is consistent in all patterns.
#Bits 8-11 (AOD QA): Must be 0000 (Best) - This is consistent in all patterns.
#Bit 12 (No glint): Must be 0 - This is consistent in all patterns.
#Bits 13-14 (Aerosol Model): Can be 00 or 01 - Your patterns cover both.
#Bit 15 (Reserved): Must be 0 - This is consistent in all patterns.
#The only varying parts in your criteria are bits 3-4 (Land/Water) and bits 13-14 (Aerosol Model).
#Each of these can have two possible values, resulting in 2 * 2 = 4 combinations.
#Your four patterns precisely cover these four possibilities:
#Land (00) with Aerosol Model 00
#Land (00) with Aerosol Model 01
#Water (01) with Aerosol Model 00
#Water (01) with Aerosol Model 01

In [66]:
#Now lets find the corresponding decimal for this Desired 16 bit binaries 
def binary_to_decimal(binary_string):
    return int(binary_string, 2)

# Example usage
binary_strings = ['0000000000000001', '0000000000001001', '0010000000000001', '0010000000001001']
decimal_values = [binary_to_decimal(b) for b in binary_strings]

for binary, decimal in zip(binary_strings, decimal_values):
    print(f"The decimal value of {binary} is: {decimal}")

The decimal value of 0000000000000001 is: 1
The decimal value of 0000000000001001 is: 9
The decimal value of 0010000000000001 is: 8193
The decimal value of 0010000000001001 is: 8201


In [67]:
#So AODQA across all the 4 orbits with the values 1,9,8193 and 8210 represent best Quality pixel. 
#Now we will correspondingly match the position with AOD550 and AOD470 for these positions, i.e for any position of AODQA other than 
## those whose values are 1,9,8193 and 8210 are in-validated or not used 
valid_values = {1, 9, 8193, 8201}
# Create a mask of valid QA values
valid_mask = np.isin(aodqa, list(valid_values))
# Create copies of aod550 and aod470 to store the filtered results
aod550_filtered = np.where(valid_mask, aod550, np.nan)
aod470_filtered = np.where(valid_mask, aod470, np.nan)
aod550_filtered.shape
aod470_filtered.shape

(4, 1200, 1200)

In [73]:
# Calculate daily average AOD across all orbits
daily_aod550 = np.nanmean(aod550_filtered, axis=0)
daily_aod470 = np.nanmean(aod470_filtered, axis=0)
# Check the shapes of the resulting arrays
print(f"Shape of daily_aod550: {daily_aod550.shape}")
print(f"Shape of daily_aod470: {daily_aod470.shape}")

Shape of daily_aod550: (1200, 1200)
Shape of daily_aod470: (1200, 1200)


/var/folders/_k/q3nwf72n5l7c_fm7sh7pzls40000gn/T/ipykernel_77679/1460969127.py:2: RuntimeWarning: Mean of empty slice
  daily_aod550 = np.nanmean(aod550_filtered, axis=0)
/var/folders/_k/q3nwf72n5l7c_fm7sh7pzls40000gn/T/ipykernel_77679/1460969127.py:3: RuntimeWarning: Mean of empty slice
  daily_aod470 = np.nanmean(aod470_filtered, axis=0)


In [ ]:
## Please Read ##
#MODIS uses a fixed grid system for its data products.
#Each pixel in the grid represents the same geographical location across different orbits.
#The sinusoidal projection used by MODIS ensures that pixel sizes remain constant across the image.


In [83]:
#Extract structure attribute from metadata#
fattrs = hdf.attributes(full=1)
ga = fattrs["StructMetadata.0"]
gridmeta = ga[0]
#Extract upper left and lower right coordinates
ul_regex = re.compile(r'''UpperLeftPointMtrs=\(
                          (?P<upper_left_x>[+-]?\d+\.\d+)
                          ,
                          (?P<upper_left_y>[+-]?\d+\.\d+)
                          \)''', re.VERBOSE)

match = ul_regex.search(gridmeta)
x0 = np.double(match.group('upper_left_x'))
y0 = np.double(match.group('upper_left_y'))

lr_regex = re.compile(r'''LowerRightMtrs=\(
                          (?P<lower_right_x>[+-]?\d+\.\d+)
                          ,
                          (?P<lower_right_y>[+-]?\d+\.\d+)
                          \)''', re.VERBOSE)
match = lr_regex.search(gridmeta)
x1 = np.double(match.group('lower_right_x'))
y1 = np.double(match.group('lower_right_y'))

# Get dimensions of stacked array
#Create Grid Coordinates
time_array, ny, nx = aod550.shape
x = np.linspace(x0, x1, nx)
y = np.linspace(y0, y1, ny) 
xv, yv = np.meshgrid(x, y)
# Initialize projection transforms
# Sinusoidal projection used by the MODIS data to WGS84 latitude/longitude projection
sinu = pyproj.Proj("+proj=sinu +R=6371007.181 +nadgrids=@null +wktext")
wgs84 = pyproj.Proj("+init=EPSG:4326")

# Tile the meshgrids to match stacked data shape
tiled_xv = np.tile(xv, (num_orbits, 1, 1))
tiled_yv = np.tile(yv, (num_orbits, 1, 1))

# Project the tiled grids  
lon, lat = pyproj.transform(sinu, wgs84, tiled_xv, tiled_yv)

# Adjust longitude  
lon[lon < 0] += 360
lon_flat = lon.flatten()
lat_flat = lat.flatten()
# Now we can check if latitudes and longitudes for each orbit are the same
lat_same = all(np.array_equal(lat, lat) for _ in range(4))
lon_same = all(np.array_equal(lon, lon) for _ in range(4))

print(f"Are all latitude arrays the same? {lat_same}")
print(f"Are all longitude arrays the same? {lon_same}")
#Extract 2D Latitude and Longitude
lat_2d = lat[0, :, :]
lon_2d = lon[0, :, :]

/Users/pravash/anaconda3/envs/test_env/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/_k/q3nwf72n5l7c_fm7sh7pzls40000gn/T/ipykernel_77679/1443067224.py:41: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = pyproj.transform(sinu, wgs84, tiled_xv, tiled_yv)


Are all latitude arrays the same? True
Are all longitude arrays the same? True


Date: 2018-04-30
Type of date: <class 'datetime.date'>


In [114]:
import netCDF4 as nc
import numpy as np
from datetime import datetime, timedelta, date
time = np.datetime64(time_array[0], 'D').astype(datetime)
print(f"Date: {time}")
print(f"Type of date: {type(time)}")
# Create a NetCDF file
filename = f"AOD_data_{time.strftime('%Y%m%d')}.nc"
dataset = nc.Dataset(filename, 'w', format='NETCDF4')
# Create dimensions
dataset.createDimension('time', 1)  # We're dealing with daily data, so time dimension is 1
dataset.createDimension('latitude', 1200)
dataset.createDimension('longitude', 1200)

# Create variables
times = dataset.createVariable('time', 'f8', ('time',))
latitudes = dataset.createVariable('latitude', 'f4', ('latitude',))
longitudes = dataset.createVariable('longitude', 'f4', ('longitude',))
aod550 = dataset.createVariable('aod550', 'f4', ('time', 'latitude', 'longitude',))
aod470 = dataset.createVariable('aod470', 'f4', ('time', 'latitude', 'longitude',))

# Add attributes
dataset.description = 'Daily AOD data'
dataset.history = f'Created {datetime.now()}'
dataset.source = 'MODIS data'

times.units = 'days since 1970-01-01 00:00:00'
times.calendar = 'gregorian'

latitudes.units = 'degrees_north'
longitudes.units = 'degrees_east'

aod550.units = 'dimensionless'
aod550.long_name = 'Aerosol Optical Depth at 550nm'

aod470.units = 'dimensionless'
aod470.long_name = 'Aerosol Optical Depth at 470nm'
# Write data
times[0] = (time - date(1970, 1, 1)).days
latitudes[:] = lat_2d[:, 0]  # Assuming lat_2d is the same for all longitudes
longitudes[:] = lon_2d[0, :]  # Assuming lon_2d is the same for all latitudes
aod550[0, :, :] = daily_aod550
aod470[0, :, :] = daily_aod470

# Close the dataset
dataset.close()

print(f"NetCDF file '{filename}' has been created successfully.")

Date: 2018-04-30
Type of date: <class 'datetime.date'>
NetCDF file 'AOD_data_20180430.nc' has been created successfully.
